In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

Using TensorFlow backend.


In [2]:
from PIL import Image

In [3]:

ds = pydicom.dcmread('test1.dcm')   

In [4]:
ds.StudyDescription

'No Finding'

In [52]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('\n> Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)    
    print('  Finding: {}'.format(ds.StudyDescription))
    img = ds.pixel_array
    
    if (ds.Modality, ds.BodyPartExamined)==('DX', 'CHEST') and ds.PatientPosition in ['PA','AP']:
        img = ds.pixel_array
        return img
    print('  Skipping file {} : \n- Modality: \"{}\"\n- Body Part: \"{}\"\n- Patient Position: \"{}\"'
          .format(filename, ds.Modality, ds.BodyPartExamined, ds.PatientPosition))
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size, rescale=1/255): 
    proc_img = img * rescale
    proc_img = (img - img_mean) / img_std
    proc_img.resize(img_size, refcheck=True)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path, json=True):
    if json:
        with open(model_path, 'r') as f:
            jmodel = f.read()
            model = keras.models.model_from_json(jmodel)
    else:
        model = keras.models.load_model(model_path)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    p_Y = model.predict(img)
    print(p_Y)
    prediction = 'Pneumonia' if p_Y[0,0] >= thresh else 'No pneumonia'
    return prediction 

In [54]:


model_path = 'latest_model.json'#path to saved model
weight_path = "{}_my_model.best.hdf5".format('xray_class')#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

my_model = load_model(model_path, weight_path) #loads model

thresh = 0.25 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
    img_mean = img.mean()# the self mean of each image was used during training
    img_std = img.std()# the self std dev of each image was used during training
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)


> Load file test1.dcm ...
  Finding: No Finding
(1, 224, 224, 3)
[[0.53022355]]
Pneumonia

> Load file test2.dcm ...
  Finding: Cardiomegaly
(1, 224, 224, 3)
[[0.54142046]]
Pneumonia

> Load file test3.dcm ...
  Finding: Effusion
(1, 224, 224, 3)
[[0.5366784]]
Pneumonia

> Load file test4.dcm ...
  Finding: No Finding
  Skipping file test4.dcm : 
- Modality: "DX"
- Body Part: "RIBCAGE"
- Patient Position: "PA"

> Load file test5.dcm ...
  Finding: No Finding
  Skipping file test5.dcm : 
- Modality: "CT"
- Body Part: "CHEST"
- Patient Position: "PA"

> Load file test6.dcm ...
  Finding: No Finding
  Skipping file test6.dcm : 
- Modality: "DX"
- Body Part: "CHEST"
- Patient Position: "XX"


The above results can be used within a PACS workstation for workflow prioritization.